In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GEMINI_API_Key']=os.getenv("GEMINI_API_Key")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data INgestion by Langchain and webapp
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this content, it might be helpful to read the following:Data Retention Conceptual DocsUsage Limiting Conceptual DocsnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of billing. If you are on Enterprise plan a

In [22]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
chunks


[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this co

I0000 00:00:1722278459.236183 2924810 subchannel.cc:806] subchannel 0x176f5a770 {address=ipv6:%5B2404:6800:4002:814::200a%5D:443, args={grpc.client_channel_factory=0x10b1ffff0, grpc.default_authority=generativelanguage.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x176f507f0, grpc.internal.client_channel_call_destination=0x1197be718, grpc.internal.event_engine=0x176f5bf80, grpc.internal.security_connector=0x176f598f0, grpc.internal.subchannel_pool=0x11a19b300, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/1.65.1, grpc.resource_quota=0x11a197450, grpc.server_uri=dns:///generativelanguage.googleapis.com:443}}: connect failed (UNKNOWN:connect: No route to host (65) {created_time:"2024-07-30T00:10:59.235684+05:30"}), backing off for 1000 ms
I0000 00:00:1722278459.492891 2924806 subchannel.cc:806] subchannel 0x176f74d80 {address=ipv6:%5B2404:6800:4002:815::200a%5D:443,

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.getenv("GEMINI_API_Key"))


I0000 00:00:1722278316.016315 2924223 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [14]:
# specify embedding model (using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  #model_kwargs=model_kwargs
)

In [23]:
# Vectore Store

from langchain_community.vectorstores import FAISS

vectorstoredb=FAISS.from_documents(chunks,embedding)

In [24]:
vectorstoredb

In [31]:
# Query with Vectorstore db

query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
print(result[0].page_content)

That's a cost reduction of nearly 75% per day!Optimization 2: limit usage​In the previous section, we managed data retention settings to optimize existing spend. In this section, we will
use usage limits to prevent future overspend.LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we've
been tracking on our usage graph. We can use these in tandem to have granular control over spend.To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the
bottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along
with a cost estimate:Lets start by setting limits on our production usage, since that is where the majority of spend comes from.Setting a good total traces limit​Picking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You should


In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GEMINI_API_Key"))


I0000 00:00:1722278947.280063 2924223 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722278947.280671 2924223 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [39]:
# Retrival chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [36]:
tempplate =  """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""


In [43]:
prompt =ChatPromptTemplate.from_template(tempplate)

# Create stuff document

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
| ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x17a280580>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x17a277e80>, default_metadata=())
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [44]:
# Create Retriver

retriver = vectorstoredb.as_retriever()

In [46]:
# retrival chain

from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, document_chain)

In [47]:
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x17a22c3d0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
            | ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.Genera

In [49]:
## Get the response form the LLM
response=retrival_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

I0000 00:00:1722279345.192538 2938145 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1722279345.193498 2938145 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


'To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the bottom of the page that lets you set usage limits per workspace.'

In [50]:
print(response['answer'])

To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the bottom of the page that lets you set usage limits per workspace.
